In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('/Users/yuyizhou/Documents/2017Fall/CMPT741/project/data/train_rating.txt', parse_dates=True)
df = df[['user_id','business_id', 'date', 'rating']]

In [3]:
num_of_u = len(df['user_id'].unique())
num_of_b = len(df['business_id'].unique())

In [4]:
u_start = 0
b_start = u_start + num_of_u
dow_start = b_start + num_of_b
moy_start = dow_start + 7
dom_start = moy_start + 12
woy_start = dom_start + 31
doy_start = woy_start + 52

In [5]:
def convert_category_type(id, start):
    return str(start+id) + ':1'

In [6]:
def add_time(df_t):
    df = df_t.copy()
    df['date'] = pd.to_datetime(df['date'])
    df['dow'] = df['date'].dt.dayofweek
    df['doy'] = df['date'].dt.dayofyear
    df['woy'] = df['date'].dt.weekofyear
    df['moy'] = df['date'].dt.month
    df['dom'] = df['date'].dt.day    
    return df

In [7]:
def add_libFM_features(df_t):
    df = df_t.copy()
    df['uid_c'] = df['user_id'].apply(lambda x: convert_category_type(x, u_start))
    df['bid_c'] = df['business_id'].apply(lambda x: convert_category_type(x, b_start))
    df['dow_c'] = df['dow'].apply(lambda x: convert_category_type(x, dow_start))
    df['moy_c'] = df['moy'].apply(lambda x: convert_category_type(x, moy_start))
    df['dom_c'] = df['dom'].apply(lambda x: convert_category_type(x, dom_start))
    df['woy_c'] = df['woy'].apply(lambda x: convert_category_type(x, woy_start))
    df['doy_c'] = df['doy'].apply(lambda x: convert_category_type(x, doy_start))
    return df

In [8]:
columns = ['rating','uid_c', 'bid_c', 'dow_c', 'moy_c', 'dom_c', 'woy_c', 'doy_c']

In [13]:
#train, test = train_test_split(df, test_size=0.2, random_state=888)
# instead of random split, here choose validation set contains only month from Jan to July
df['date'] = pd.to_datetime(df['date'])
df['MOY'] = df['date'].dt.month
test = (df.loc[(df['MOY'] >= 1) & (df['MOY'] <= 7)]) \
     .sample(frac = 0.3493389, random_state = np.random.seed(5)) \
     .drop(['MOY'], 1)
train = df.loc[~df.index.isin(test.index)] 

In [14]:
len(train)

1630504

In [15]:
len(test)

407626

In [16]:
train = add_time(train)
train = add_libFM_features(train)
train = train[columns]

In [17]:
train.head()

,rating,uid_c,bid_c,dow_c,moy_c,dom_c,woy_c,doy_c
0,5,0:1,693209:1,838514:1,838530:1,838551:1,838609:1,838938:1
3,3,3:1,693210:1,838517:1,838526:1,838542:1,838590:1,838806:1
5,2,5:1,693210:1,838514:1,838525:1,838546:1,838586:1,838780:1
6,4,6:1,693210:1,838514:1,838527:1,838541:1,838594:1,838836:1
7,2,7:1,693210:1,838512:1,838525:1,838548:1,838587:1,838782:1


In [18]:
train.to_csv('/Users/yuyizhou/Documents/2017Fall/CMPT741/project/libFM/data/train7.libfm', sep=' ', header=False, index=False)

In [19]:
test = add_time(test)
test = add_libFM_features(test)
test = test[columns]

In [20]:
test.head()

,rating,uid_c,bid_c,dow_c,moy_c,dom_c,woy_c,doy_c
1518715,1,575629:1,816322:1,838516:1,838526:1,838553:1,838591:1,838818:1
173638,5,111672:1,709914:1,838515:1,838524:1,838546:1,838582:1,838749:1
1688893,4,414425:1,823259:1,838516:1,838526:1,838544:1,838590:1,838809:1
624778,3,315419:1,755864:1,838517:1,838525:1,838535:1,838584:1,838769:1
169709,1,118952:1,709557:1,838514:1,838524:1,838549:1,838582:1,838752:1


In [21]:
test.to_csv('/Users/yuyizhou/Documents/2017Fall/CMPT741/project/libFM/data/test7.libfm', sep=' ', header=False, index=False)